# State change propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [2]:
class SubModel(bu.Model):
    name = 'submodel'
    length = bu.Float(2, GEO=True)

    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('submodel changed')

    ipw_view = bu.View(
        bu.Item('length')
    )

In [3]:
sm = SubModel(state_change_debug=True)

In [4]:
sm.length = 3

state_changed <__main__.SubModel object at 0x7f632fc66f90> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632fc66f90>, name='length', old=2, new=3.0)
submodel changed


The change of a model parameter is observed and notified
by an event `sm.state_changed`. Let us observe this event
using a function


After executing the above cell, change the length in the
above model window to see the spy in action.

How did it happen? The `length` parameter of the `SubModel`
contains the metadate `GEO=True`. This indicates the type of
model input data. Except of `GEO` there are also the following
types of input data:

 - `BC` - boundary condition
 - `MAT` - material parameter
 - `DSC` - discretization
 - `CS` - cross section
 - `ALG` - algorithm parameter
 - `GEO` - global geometry

Change of any parameter denoted by one of the above tags
is observed and propagated along the model tree.

Let us define a more complex model

In [5]:
class InterimModel(bu.Model):
    sm = bu.Instance(SubModel,(), GEO=True)
    stiffness = bu.Float(400, GEO=True)

    tree = ['sm']
    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('interim model changed')


In [6]:
im = InterimModel(state_change_debug=True)

sub_name sm <__main__.InterimModel object at 0x7f632f079090>


In [7]:
im.sm.length = 7

submodel changed
state_changed <__main__.InterimModel object at 0x7f632f079090> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632f962590>, name='state_changed', old=<undefined>, new=True)
interim model changed
sub_name sm <__main__.InterimModel object at 0x7f632f079090>


In [8]:
class TopLevelModel(bu.Model):
    im = bu.Instance(InterimModel,(), DSC=True)
    time = bu.Float(400, ALG=True)

    tree = ['im']
    @tr.observe('state_changed')
    def toplevel_changed(self, event):
        print('toplevel model changed')



In [9]:
tl = TopLevelModel(state_change_debug=True)
tl.im.state_change_debug = True
tl.im.sm.state_change_debug = True

sub_name sm <__main__.InterimModel object at 0x7f632f028590>
sub_name im <__main__.TopLevelModel object at 0x7f632fce91d0>


In [10]:
tl.im.sm.length = 10

state_changed <__main__.SubModel object at 0x7f632f79ca40> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632f79ca40>, name='length', old=2, new=10.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f632f028590> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632f79ca40>, name='state_changed', old=<undefined>, new=True)
interim model changed
sub_name sm <__main__.InterimModel object at 0x7f632f028590>
state_changed <__main__.TopLevelModel object at 0x7f632fce91d0> TraitChangeEvent(object=<__main__.InterimModel object at 0x7f632f028590>, name='state_changed', old=<undefined>, new=True)
toplevel model changed
sub_name im <__main__.TopLevelModel object at 0x7f632fce91d0>


# Test a list

In [11]:
class CrossSectionLayout(bu.ModelList):
    name = 'Cross Section Layout'

    items = bu.List(SubModel, [])

    def add_layer(self, rl):
        self.items.append(rl)

In [12]:
csl = CrossSectionLayout(state_change_debug=True)

In [13]:
csl.add_layer(SubModel(state_change_debug=True))

In [14]:
csl.update_observers()

sub_name submodel <__main__.CrossSectionLayout object at 0x7f632f167180>


In [18]:
csl.items[0].length = 3

state_changed <__main__.SubModel object at 0x7f632f6fe130> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632f6fe130>, name='length', old=4.0, new=3.0)
submodel changed
state_changed <__main__.CrossSectionLayout object at 0x7f632f167180> TraitChangeEvent(object=<__main__.SubModel object at 0x7f632f6fe130>, name='state_changed', old=<undefined>, new=True)
sub_name submodel <__main__.CrossSectionLayout object at 0x7f632f167180>


In [19]:
csl.as_node('root')

('root',
 [('submodel', <__main__.SubModel at 0x7f632f6fe130>, [])])